In [1]:
# !pip install beautifulsoup4
# !pip install hazm

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
from hazm import *

# Last names data preparation

## Data crawling for last name

### wikipedia

In [44]:
url = 'https://fa.wikipedia.org/w/index.php?title=%D8%B1%D8%AF%D9%87:%D9%86%D8%A7%D9%85%E2%80%8C%D9%87%D8%A7%DB%8C_%D8%AE%D8%A7%D9%86%D9%88%D8%A7%D8%AF%DA%AF%DB%8C&pageuntil=%D8%AA%D8%B1%DB%8C%D8%A7%D9%86#mw-pages'
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

In [45]:
last_names_list = list()

In [46]:
while True:
    f = requests.get(url, headers = headers)
    soup = BeautifulSoup(f.content, 'lxml')
    last_name_part = soup.find_all('div', {'id': 'mw-pages'})[0]
    last_names_lis = last_name_part.find_all('li')
    for last_name in last_names_lis:
        text = last_name.find("a").get_text()
        if not "(نام)" in text:
            last_names_list.append(last_name.find("a").get_text().split(" (")[0])
    all_links = last_name_part.find_all("a", href= True)
    end = True
    for link in all_links:
        if link.get_text() == 'صفحهٔ بعدی':
            url = "https://fa.wikipedia.org" + link['href']
            end = False
            break
    if end:
        break

### itel

In [50]:
url = 'http://www.itel.ir/page133.aspx'
f = requests.get(url, headers = headers)
soup = BeautifulSoup(f.content, 'lxml')

In [58]:
persian_text_prts = soup.find_all('div', {'class' : 'centerboxm'})[0].find_all('span', {'lang' : 'FA'})
last_names_list2 = list()
for persian_text_part in persian_text_prts:
    last_names_list2.append(persian_text_part.get_text())

In [65]:
last_names_list.extend(last_names_list2[5:-5])

In [71]:
len(last_names_list)

1043

## Writing data in json 

In [72]:
file = open('last_names.json', 'w')
json.dump(last_names_list, file, indent = 6)
file.close()

# First names data preparation

In [82]:
df = pd.read_csv('first_names.csv', index_col=False)

normalizer = Normalizer()

In [90]:
first_names_list = list()
for i in range(len(df['first_name'])):
    first_names_list.append(normalizer.normalize(df['first_name'][i]))

## Writing data in json 

In [92]:
file = open('first_names.json', 'w')
json.dump(first_names_list, file, indent = 6)
file.close()

# Loading last names data

In [98]:
file = open('last_names.json', 'r')
last_names = json.load(file)
file.close()

last_names_reg = ""
for last_name in last_names:
    last_names_reg += normalizer.normalize(last_name) + "|"
last_names_reg = last_names_reg[:-1]

# Loading first names data

In [99]:
file = open('first_names.json', 'r')
first_names = json.load(file)
file.close()

first_names_reg = ""
for first_name in first_names:
    first_names_reg += first_name + "|"
first_names_reg = first_names_reg[:-1]

In [164]:
file = open('last_names_regex.json', 'w')
json.dump(last_names_reg, file, indent = 6)
file.close()

file = open('first_names_regex.json', 'w')
json.dump(first_names_reg, file, indent = 6)
file.close()